In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import pandas as pd
from pathlib import Path
import json
import locale
from IPython.display import display
from datetime import datetime
from pandas.tseries.offsets import MonthEnd

from helpers import rename_postaja, simplify_column, lower_sumniki
from functools import reduce

Step1: DataFrame for each air pollutant

- read each csv
- input into dataframe
- convert into long format


In [79]:
NEVARNE_SNOVI = ["SO2", "PM10", "NO2"]
NEVARNE_SNOVI_p = ["SO2", "PM10"]
MESECI = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "Maj",
    "Jun",
    "Jul",
    "Avg",
    "Sep",
    "Okt",
    "Nov",
    "Dec",
]

YEAR_MONTH_SORT = {
    "by": ["Leto", "Mesec"],
    "key": lambda col: (
        col
        if col.name != "Mesec"
        else col.map(lambda x: datetime.strptime(x, "%b").month)
    ),
}


def presezki_mapper(snov: str, columns: list[str]):
    full_mapper = {
        "Ura presežek": f"{snov} - Urnih presežkov",
        "Dan presežek": f"{snov} - Dnevnih presežkov",
    }
    return {k: v for k, v in full_mapper.items() if k in columns}

In [ ]:
def dict_to_csv(repo: dict[str, pd.DataFrame], form: str, cas: str):
    for snov, df in repo.items():
        path_dir = Path(f"podatki/df_{form}/{cas}")

        if not Path.exists(path_dir):
            path_dir.mkdir()

        filepath = path_dir / f"{snov}.csv"
        df.to_csv(filepath, index=False)
        print(f"saved: {filepath}")


def load_air_pollutants(nevarne_snovi: list[str], cas: str) -> dict[str, pd.DataFrame]:
    pd_list = []

    for i, snov in enumerate(nevarne_snovi):
        dir_path = Path(f"podatki/{snov}/{cas}")
        if not dir_path.exists():
            continue

        tmp_list = []
        file_list = sorted(x for x in dir_path.iterdir())

        for file_path in file_list:
            leto = file_path.stem
            # print(leto)

            df = pd.read_csv(file_path, na_values=["", ",", " ", "/"])

            # cleanup
            df = rename_postaja(df)
            df = simplify_column(df, "Postaja", True)

            # insert year column
            df.insert(1, "Leto", leto)
            # wide to long conversion

            if cas in ["mesecne", "urne", "dnevne"]:
                df = df.melt(
                    id_vars=["Postaja", "Leto"],
                    value_vars=MESECI,
                    var_name="Mesec",
                    value_name=snov,
                )
            elif cas == "presezki":
                columns = [
                    col for col in ["Ura presežek", "Dan presežek"] if col in df.columns
                ]

                df = df.loc[~(df[columns] == 0).all(axis=1)]

                df.rename(
                    columns=presezki_mapper(snov, columns),
                    inplace=True,
                )

            # append to tmp list
            tmp_list.append(df)
        pd_list.append(pd.concat(tmp_list,ignore_index=True))

    return pd_list


mesecne_meritve = load_air_pollutants(NEVARNE_SNOVI, "mesecne")
urne_meritve = load_air_pollutants(NEVARNE_SNOVI, "urne")
dnevne_meritve = load_air_pollutants(NEVARNE_SNOVI, "dnevne")


,Postaja,Leto,Mesec,NO2
0,ljubljana f.,1997,Jan,31.0
1,maribor,1997,Jan,45.0
2,celje,1997,Jan,42.0
3,trbovlje,1997,Jan,NaN
4,zavodnje,1997,Jan,15.0
...,...,...,...,...
4663,mb tezno,2023,Dec,28.0
4664,skale,2023,Dec,10.0
4665,sostanj,2023,Dec,14.0
4666,zavodnje,2023,Dec,6.0


Step2: Map stations to regions


In [ ]:
# returns (postaja->regija map, missing postaja)
def postaja_regija_map(slovar_path: str) -> pd.Series:
    # load dict from file
    regija_dict = json.load(open(slovar_path))
    # create dict for postaja->regija
    regija_dict = {
        postaja: regija
        for regija, postaje in regija_dict.items()
        for postaja in postaje
    }
    # create series for mapper; cleanup names, drop duplicates
    regija_series = pd.Series(regija_dict, name="Regija")
    regija_series.index = regija_series.index.map(lower_sumniki)
    regija_series = regija_series[~regija_series.index.duplicated()]
    return regija_series


# check and print if postaja exists without mapping
def check_missing_mapping(postaje_dict: dict[str, pd.DataFrame], map_regija: pd.Series):
    prevodi = set(map_regija.index.unique())
    for snov, df in postaje_dict.items():
        postaje = set(df["Postaja"].unique())
        inter = postaje - prevodi
        if len(inter) > 0:
            print(f"{snov}: {inter}")


map_regija = postaja_regija_map("podatki/slovar.json")

check_missing_mapping(mesecne_meritve, map_regija)
check_missing_mapping(urne_meritve, map_regija)
check_missing_mapping(dnevne_meritve, map_regija)

In [92]:
def add_regije_df(
    postaje_dict: dict[str, pd.DataFrame], mapper: pd.Series
) -> dict[str, pd.DataFrame]:
    regije_dict = {}
    for snov, df in postaje_dict.items():
        regija_df = df.copy()
        regija_df.insert(loc=1, column="Regija", value=regija_df["Postaja"].map(mapper))

        regije_dict[snov] = regija_df

    return regije_dict


reg_mesecne_meritve = add_regije_df(postaje_dict=mesecne_meritve, mapper=map_regija)
reg_urne_meritve = add_regije_df(postaje_dict=urne_meritve, mapper=map_regija)
reg_dnevne_meritve = add_regije_df(postaje_dict=dnevne_meritve, mapper=map_regija)

reg_mesecne_meritve.values()

dict_values([           Postaja             Regija  Leto Mesec    SO2
0     ljubljana f.  Osrednjeslovenska  1997   Jan   50.0
1     ljubljana b.  Osrednjeslovenska  1997   Jan   69.0
2          maribor          Podravska  1997   Jan   50.0
3            celje          Savinjska  1997   Jan   72.0
4         trbovlje           Zasavska  1997   Jan  106.0
...            ...                ...   ...   ...    ...
6823     topolsica          Savinjska  2023   Dec    2.0
6824       velenje          Savinjska  2023   Dec    3.0
6825    veliki vrh          Savinjska  2023   Dec    5.0
6826      zavodnje          Savinjska  2023   Dec    6.0
6827   mobilna tes          Savinjska  2023   Dec    4.0

[6828 rows x 5 columns],             Postaja             Regija  Leto Mesec  PM10
0      ljubljana f.  Osrednjeslovenska  1999   Jan  43.0
1             celje          Savinjska  1999   Jan  47.0
2          trbovlje           Zasavska  1999   Jan  52.0
3         eis celje          Savinjska  1999   Ja

In [ ]:
locale.setlocale(locale.LC_ALL, "sl_SI.utf8")

df_mesecne = reduce(
    lambda left,right: pd.merge(left,right,on=)
)

In [ ]:
PM10 = mesecne_regije_dict["PM10"].copy()
SO2 = mesecne_regije_dict["SO2"].copy()
NO2 = mesecne_regije_dict["NO2"].copy()
NOX = mesecne_regije_dict["NOX"].copy()


tmp_list = [PM10, SO2, NO2, NOX]

df_wide = reduce(
    lambda left, right: pd.merge(
        left, right, on=["Postaja", "Regija", "Leto", "Mesec"], how="outer"
    ),
    tmp_list,
)
df_wide.sort_values(**YEAR_MONTH_SORT, inplace=True)
df_wide.insert(
    loc=2,
    column="Datum",
    value=pd.to_datetime(
        df_wide["Leto"].astype(str) + "-" + df_wide["Mesec"].astype(str), format="%Y-%b"
    )
    + MonthEnd(0),
)
df_wide.drop(columns=["Leto", "Mesec"], inplace=True)

df_wide.to_csv("podatki/wide_df.csv", index=False)

df_wide

In [ ]:
PM10_long = PM10.rename(columns={"PM10": "Vrednost"})
PM10_long.insert(loc=4, column="Snov", value="PM10")

SO2_long = SO2.rename(columns={"SO2": "Vrednost"})
SO2_long.insert(loc=4, column="Snov", value="SO2")

NO2_long = NO2.rename(columns={"NO2": "Vrednost"})
NO2_long.insert(loc=4, column="Snov", value="NO2")

NOX_long = NOX.rename(columns={"NOX": "Vrednost"})
NOX_long.insert(loc=4, column="Snov", value="NOX")

tmp_list = [PM10_long, SO2_long, NO2_long, NOX_long]

long_df = pd.concat(tmp_list)
long_df.sort_values(**YEAR_MONTH_SORT, inplace=True)

# long_df.to_csv("podatki/long_df.csv", index=False)